In [1]:
from IPython.display import HTML

"""Cash ladder

Demonstration of how to compute a cash ladder for a portfolio.

Attributes
----------
cash
"""

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz
from cashladder import get_cash_ladder
#from get_cashforecast import cash_forecast

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [4]:
with open("config/cash_ladder_seed_data.json", "r") as seed_data_config:
    mapping_file = json.loads(seed_data_config.read())
    
# Create a new scope

scope = "cash-ladder-001"
portfolio_code = "EQUITY_FUND"

# Load a file of equity transactions

transactions_file = r"data/cash_ladder.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
    mappings = mapping_file
)

# Enter details to run the LUSID Cashladder

In [5]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

# defining some widgets
portfolio_text_box = widgets.Text(
       value='EQUITY_FUND',
       description='Portfolio:', )


settlement_date = widgets.DatePicker(
    value= datetime(year=2020, month=4, day=1, tzinfo=pytz.UTC),
           description='Date:')

ccy_menu = widgets.Dropdown(
       options=['GBP', 'USD'],
       description='Currency:')


box = widgets.VBox([portfolio_text_box,settlement_date, ccy_menu])

# defining a list with the contents of our windows
children = [box]
# initializing a tab
tab = widgets.Tab()
# setting the tab windows 
tab.children = children
# changing the title of the first and second window
tab.set_title(0, 'Select details:')
tab

In [6]:
btn = widgets.Button(description='Cashladder')

display(btn)

def highlight_col(x):
    #copy df to new - original data are not changed
    df = x.copy()
    #set by condition
    mask = df['CashLadderStatus'] == "SOD Balance"
    df.loc[mask, :] = 'color: red'
    df.loc[~mask,:] = 'color: ""'
    return df    


def btn_eventhandler(obj):
    
    print("You selected portfolio: ", portfolio_text_box.value)
    print("You selected date: ", settlement_date.value)
    print("You selected currency: ", ccy_menu.value)
    
    df = get_cash_ladder(settlement_date.value, portfolio_text_box.value, ccy_menu.value, scope)
    df["CashImpact"] = df["CashImpact"].apply(lambda x: int(x))
    df = df.style.apply(highlight_col, axis=None)
    
    
    display(df)
    
btn.on_click(btn_eventhandler)

Button(description='Cashladder', style=ButtonStyle())

You selected portfolio:  EQUITY_FUND
You selected date:  2020-04-02
You selected currency:  GBP


,ReportRunTime,SettlementDate,Currency,Portfolio,CashLadderStatus,InstrumentName,CashImpact,TransactionId,TradeStatus
0,16:21:47,2020-04-02,GBP,EQUITY_FUND,SOD Balance,,10300000,,
2,16:21:47,2020-04-02,GBP,EQUITY_FUND,EOD Balance,,10300000,,
3,16:21:47,2020-04-03,GBP,EQUITY_FUND,SOD Balance,,10300000,,
4,16:21:47,2020-04-03,GBP,EQUITY_FUND,Cash impact from a Buy transaction,vodafone,-1000000,trd_001,confirmed
5,16:21:47,2020-04-03,GBP,EQUITY_FUND,Cash impact from a Buy transaction,vodafone,-2000000,trd_002,confirmed
6,16:21:47,2020-04-03,GBP,EQUITY_FUND,Cash impact from a Sell transaction,vodafone,100000,trd_003,confirmed
7,16:21:47,2020-04-03,GBP,EQUITY_FUND,Cash impact from a Buy transaction,Anglo American plc,-1000000,trd_004,confirmed
8,16:21:47,2020-04-03,GBP,EQUITY_FUND,Cash impact from a Buy transaction,Anglo American plc,-3000000,trd_005,confirmed
9,16:21:47,2020-04-03,GBP,EQUITY_FUND,Cash impact from a FundsIn transaction,CCY_GBP,2000000,cash_001,confirmed
10,16:21:47,2020-04-03,GBP,EQUITY_FUND,Cash impact from a FundsOut transaction,CCY_GBP,-3000000,cash_003,confirmed


You selected portfolio:  EQUITY_FUND
You selected date:  2020-04-02
You selected currency:  USD


,ReportRunTime,SettlementDate,Currency,Portfolio,CashLadderStatus,InstrumentName,CashImpact,TransactionId,TradeStatus
0,16:21:54,2020-04-02,USD,EQUITY_FUND,SOD Balance,,17000000,,
2,16:21:54,2020-04-02,USD,EQUITY_FUND,EOD Balance,,17000000,,
3,16:21:54,2020-04-03,USD,EQUITY_FUND,SOD Balance,,17000000,,
4,16:21:54,2020-04-03,USD,EQUITY_FUND,Cash impact from a Buy transaction,IBM Ltd,-3000000,trd_009,un-confirmed
5,16:21:54,2020-04-03,USD,EQUITY_FUND,Cash impact from a Sell transaction,IBM Ltd,3000000,trd_011,un-confirmed
6,16:21:54,2020-04-03,USD,EQUITY_FUND,Cash impact from a FundsOut transaction,CCY_USD,-2000000,cash_006,un-confirmed
7,16:21:54,2020-04-03,USD,EQUITY_FUND,EOD Balance,,15000000,,
8,16:21:54,2020-04-04,USD,EQUITY_FUND,SOD Balance,,15000000,,
9,16:21:54,2020-04-04,USD,EQUITY_FUND,Cash impact from a Buy transaction,IBM Ltd,-3000000,trd_012,un-confirmed
10,16:21:54,2020-04-04,USD,EQUITY_FUND,EOD Balance,,12000000,,
